In [1]:
from datetime import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras

# from IPython.core.interactiveshell import InteractiveShell
# import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
# 配置项
# # 这个要放到设置中文之前否则还是小方框
# plt.style.use("seaborn")

# # 指定默认字体 用来正常显示中文标签
# plt.rcParams['font.sans-serif'] = ['SimHei']
# # 解决保存图像是负号'-'显示为方块的问题
# plt.rcParams['axes.unicode_minus'] = False

# #全部行都能输出
# InteractiveShell.ast_node_interactivity = "all"

In [3]:
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    y = tf.cast(y, dtype=tf.int32)
    
    return x, y

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

In [5]:
batch_size = 128

db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(batch_size)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).batch(batch_size)

In [9]:
sample = next(iter(db_train))
print('sample:', sample[0].shape, sample[1].shape)

sample: (128, 32, 32, 3) (128, 1)


In [10]:
# 5个单元的卷积层
conv_layer = [
    # 第一个单元
    keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation=tf.nn.relu),
    keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    
    # 第二个单元
    keras.layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation=tf.nn.relu),
    keras.layers.Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    
    # 第三个单元
    keras.layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation=tf.nn.relu),
    keras.layers.Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    
    # 第四个单元
    keras.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu),
    keras.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
    
    # 第五个单元
    keras.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.nn.relu),
    keras.layers.Conv2D(512, kernel_size=(3, 3), padding='same', activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='same'),
]

In [14]:
# 13层神经网络 1一个输入层 5个卷积单元(10层) + 一个隐藏层 + 一个输出层
conv_net = keras.Sequential(conv_layer)
fc_net = keras.Sequential([
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(100)
])

conv_net.build(input_shape=[None, 32, 32, 3])
fc_net.build(input_shape=[None, 512])

optimizer = keras.optimizers.Adam(1e-3)

In [23]:
trainable_variables = conv_net.trainable_variables + fc_net.trainable_variables

# 由于太慢了 就不运行了
for epoch in range(30):
    for step, (x, y) in enumerate(db_train):
        with tf.GradientTape() as tap:
            # [b, 32, 32, 3] => [b, 1, 1, 512]
            out = conv_net(x)
            # [b, 1, 1, 512] => [b, 512]
            out = tf.reshape(out, [-1, 512])
            # [b, 512] => [b, 100]
            logits = fc_net(out)
            y_true = tf.one_hot(y, depth=100)
            
            loss = tf.losses.categorical_crossentropy(y_true, logits, from_logits=True)
            loss = tf.reduce_mean(loss)
            
        grads = tap.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(grads, trainable_variables))
        
        if step %100 == 0:
            print(epoch, step, 'loss:', float(loss))
        
        # 测试数据
        total_num, total_correct = 0, 0
        for x, y in db_test:
            out = conv_net(x)
            out = tf.reshape(out, [-1, 512])
            y_pred = fc_net(out)
            y_prob = tf.nn.softmax(y_pred, axis=1)
            y_pred = tf.argmax(y_prob, axis=1, output_type=tf.int32)
            y_true = tf.squeeze(y, axis=1)
            result = tf.equal(y_pred, y_true)
            correct = tf.reduce_sum(tf.cast(result, dtype=tf.int32))
            total_correct += correct
            total_num += x.shape[0]
        acc = total_correct / total_num
        print(epoch, 'acc:', acc)
            
            

0 0 loss: 4.599603652954102
0 acc: tf.Tensor(0.0078125, shape=(), dtype=float64)


KeyboardInterrupt: 

In [19]:

for x, y in db_test.take(1):
    out = conv_net(x)
    out = tf.reshape(out, [-1, 512])
    y_pred = fc_net(out)
    y_prob = tf.nn.softmax(y_pred, axis=1)
    y_pred = tf.argmax(y_prob, axis=1, output_type=tf.int32)
    y_true = tf.squeeze(y, axis=1)
    result = tf.equal(y_pred, y_true)
    print(y_pred.shape, y_true.shape)
    print(result)

(128,) (128,)
tf.Tensor(
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False], shape=(128,), dtype=bool)
